# Microsoft 365 Graph による情報取得 (C#)
- MSAL の Device Code Flow でサインイン
- Microsoft Graph のエンドポイント（/me、OneDrive 最近使用ファイル、Teams、チャット、予定表、送信済みメール）を呼び出します

In [ ]:
var clientId = "CLIENT_ID";
var tenantId = "TENANT_ID";

In [ ]:
// 依存パッケージをインストール
#r "nuget: Microsoft.Identity.Client"
#r "nuget: Microsoft.Graph"

In [ ]:
using System;
using System.Threading.Tasks;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text.Json;

using Microsoft.Identity.Client;
using Microsoft.Graph;

In [ ]:
// サインインのテスト
var scopes = new[] {
    "Files.Read",
};

// Device Code Flow でトークン取得
var publicClient = PublicClientApplicationBuilder
    .Create(clientId)
    .WithTenantId(tenantId)
    .Build();

var authResult = await publicClient.AcquireTokenWithDeviceCode(scopes, deviceCodeResult =>
{
    Console.WriteLine(deviceCodeResult.Message);
    return Task.CompletedTask;
}).ExecuteAsync();

if (authResult != null)
{
    Console.WriteLine("サインインに成功しました。");
    if (authResult.Account != null)
    {
        Console.WriteLine($"ユーザー: {authResult.Account.Username}");
    }
    Console.WriteLine($"アクセストークンの有効期限: {authResult.ExpiresOn} (UTC)");
}
else
{
    Console.WriteLine("サインインに失敗しました。");
}


In [ ]:
// 実行用にスコープを指定してトークン取得
var scopes = new[] {
    "Files.Read",
    "User.Read",
    "Mail.Read",
    "Calendars.Read",
    "Chat.Read",
    "Channel.ReadBasic.All",
    "ChannelMessage.Read.All",
    "Team.ReadBasic.All",
};

// Device Code Flow でトークン取得
var publicClient = PublicClientApplicationBuilder
    .Create(clientId)
    .WithTenantId(tenantId)
    .Build();

var authResult = await publicClient.AcquireTokenWithDeviceCode(scopes, deviceCodeResult =>
{
    Console.WriteLine(deviceCodeResult.Message);
    return Task.CompletedTask;
}).ExecuteAsync();

if (authResult != null)
{
    Console.WriteLine("サインインに成功しました。");
    if (authResult.Account != null)
    {
        Console.WriteLine($"ユーザー: {authResult.Account.Username}");
    }
    Console.WriteLine($"アクセストークンの有効期限: {authResult.ExpiresOn} (UTC)");
}
else
{
    Console.WriteLine("サインインに失敗しました。");
}


In [ ]:
// HttpClient の初期化とアクセストークンの設定
var httpClient = new HttpClient();
httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", authResult.AccessToken);

In [ ]:
// OneDrive 最近使用ファイル
async Task GetRecentFilesAsync(HttpClient client)
{
    var response = await client.GetAsync("https://graph.microsoft.com/v1.0/me/drive/recent");
    response.EnsureSuccessStatusCode();
    var json = await response.Content.ReadAsStringAsync();
    using var doc = JsonDocument.Parse(json);
    Console.WriteLine("[OneDrive Recent Files]");
    foreach (var item in doc.RootElement.GetProperty("value").EnumerateArray())
    {
        var name = item.GetProperty("name").GetString();
        var webUrl = item.GetProperty("webUrl").GetString();
        Console.WriteLine($"{name} - {webUrl}");
    }
}

// 実行
await GetRecentFilesAsync(httpClient);

In [ ]:
// Teams チャネルメッセージ＆リプライ
async Task GetTeamsChannelMessagesAsync(HttpClient client)
{
    // 所属チーム一覧取得
    var teamsRes = await client.GetAsync("https://graph.microsoft.com/v1.0/me/joinedTeams");
    teamsRes.EnsureSuccessStatusCode();
    var teamsJson = await teamsRes.Content.ReadAsStringAsync();
    using var teamsDoc = JsonDocument.Parse(teamsJson);
    foreach (var team in teamsDoc.RootElement.GetProperty("value").EnumerateArray())
    {
        var teamId = team.GetProperty("id").GetString();
        var teamName = team.GetProperty("displayName").GetString();
        Console.WriteLine($"[Team: {teamName}]");

        // チャネル一覧
        var channelsRes = await client.GetAsync($"https://graph.microsoft.com/v1.0/teams/{teamId}/channels");
        channelsRes.EnsureSuccessStatusCode();
        var channelsJson = await channelsRes.Content.ReadAsStringAsync();
        using var channelsDoc = JsonDocument.Parse(channelsJson);
        foreach (var channel in channelsDoc.RootElement.GetProperty("value").EnumerateArray())
        {
            var channelId = channel.GetProperty("id").GetString();
            var channelName = channel.GetProperty("displayName").GetString();
            Console.WriteLine($"Channel: {channelName}");
            // メッセージ一覧
            var messagesRes = await client.GetAsync($"https://graph.microsoft.com/v1.0/teams/{teamId}/channels/{channelId}/messages");
            messagesRes.EnsureSuccessStatusCode();
            var messagesJson = await messagesRes.Content.ReadAsStringAsync();
            using var messagesDoc = JsonDocument.Parse(messagesJson);
            foreach (var msg in messagesDoc.RootElement.GetProperty("value").EnumerateArray())
            {
                var msgBody = msg.GetProperty("body").GetProperty("content").GetString();
                if (msgBody != null && msgBody.Contains("<systemEventMessage/>"))
                {
                    break;
                }
                Console.WriteLine($"  Message: {msgBody}");

                // リプライ一覧
                if (msg.TryGetProperty("replies", out var replies) && replies.ValueKind == JsonValueKind.Array)
                {
                    foreach (var reply in replies.EnumerateArray())
                    {
                        var replyBody = reply.GetProperty("body").GetProperty("content").GetString();
                        if (replyBody != null && replyBody.Contains("<systemEventMessage/>"))
                        {
                            break;
                        }
                        Console.WriteLine($"    Reply: {replyBody}");
                    }
                }
            }
        }
    }
}

// 実行
await GetTeamsChannelMessagesAsync(httpClient);

In [ ]:
// Teams チャット

// チャットのメッセージを表示
async Task PrintChatMessages(HttpClient client, string chatId)
{
    var msgsResponse = await client.GetAsync($"https://graph.microsoft.com/v1.0/chats/{chatId}/messages");
    msgsResponse.EnsureSuccessStatusCode();
    var json = await msgsResponse.Content.ReadAsStringAsync();
    using var msgsDoc = JsonDocument.Parse(json);

    if (msgsDoc.RootElement.TryGetProperty("value", out var msgsArr) && msgsArr.ValueKind == JsonValueKind.Array)
    {
        foreach (var msg in msgsArr.EnumerateArray())
        {
            var fromInfo = msg.GetProperty("from");
            if (fromInfo.ValueKind == JsonValueKind.Object)
            {
                var sender = fromInfo.GetProperty("user").GetProperty("displayName").GetString() ?? "Unknown";
                var content = msg.GetProperty("body").GetProperty("content").GetString();
                var timestamp = msg.GetProperty("createdDateTime").GetDateTime();
                Console.WriteLine($"{timestamp} | {sender}: {content}");
            }
        }
    }
}

// Teams 最近のチャット
async Task GetRecentChatsAsync(HttpClient client)
{
    var response = await client.GetAsync("https://graph.microsoft.com/v1.0/me/chats");
    response.EnsureSuccessStatusCode();
    var json = await response.Content.ReadAsStringAsync();
    using var doc = JsonDocument.Parse(json);
    Console.WriteLine("[Teams Recent Chats]");
    foreach (var chat in doc.RootElement.GetProperty("value").EnumerateArray())
    {
        var chatId = chat.GetProperty("id").GetString();
        var topic = chat.TryGetProperty("topic", out var t) ? t.GetString() : "(No topic)";
        Console.WriteLine($"Chat: {topic} (ID: {chatId})");
        await PrintChatMessages(client, chatId);
        Console.WriteLine("----------");
    }
}

// 実行
await GetRecentChatsAsync(httpClient);

In [ ]:
// Outlook 会議一覧
async Task GetOutlookEventsAsync(HttpClient client)
{
    var response = await client.GetAsync("https://graph.microsoft.com/v1.0/me/events");
    response.EnsureSuccessStatusCode();
    var json = await response.Content.ReadAsStringAsync();
    using var doc = JsonDocument.Parse(json);
    Console.WriteLine("[Outlook Events]");
    foreach (var ev in doc.RootElement.GetProperty("value").EnumerateArray())
    {
        var subject = ev.GetProperty("subject").GetString();
        var start = ev.GetProperty("start").GetProperty("dateTime").GetString();
        var end = ev.GetProperty("end").GetProperty("dateTime").GetString();
        Console.WriteLine($"{subject}: {start} - {end}");
    }
}

// 実行
await GetOutlookEventsAsync(httpClient);

In [ ]:
// Outlook 送信済みメール
async Task GetOutlookSentMailsAsync(HttpClient client)
{
    var response = await client.GetAsync("https://graph.microsoft.com/v1.0/me/mailFolders/sentitems/messages");
    response.EnsureSuccessStatusCode();
    var json = await response.Content.ReadAsStringAsync();
    using var doc = JsonDocument.Parse(json);
    Console.WriteLine("[Outlook Sent Mails]");
    foreach (var mail in doc.RootElement.GetProperty("value").EnumerateArray())
    {
        var subject = mail.GetProperty("subject").GetString();
        var to = mail.GetProperty("toRecipients").EnumerateArray().Select(r => r.GetProperty("emailAddress").GetProperty("address").GetString());
        Console.WriteLine($"To: {string.Join(", ", to)} | Subject: {subject}");
    }
}

// 実行
await GetOutlookSentMailsAsync(httpClient);